# HERON Paths

In [1]:
import os
import sys
import operator
import numpy as np
import pickle
import subprocess
import shutil
import pandas as pd
import time as time_mod
from functools import partial

sys.path.append('..')

from ExternalDispatchManager import ExternalDispatchRunner

raven = "~/projects/raven/raven_framework"
heron = "~/projects/HERON/heron"
mra   = "~/projects/multiresolutionanalysis"

RAVEN_LOC = os.path.expanduser(raven)
RAVEN_DIR = os.path.abspath( os.path.join(RAVEN_LOC, '..') )
sys.path.append(RAVEN_LOC)
sys.path.append(RAVEN_DIR)

HERON_LOC = os.path.expanduser(heron)
HERON_DIR = os.path.abspath( os.path.join(HERON_LOC, '..') )
sys.path.append(HERON_LOC)
sys.path.append(HERON_DIR)

MRA_LOC = os.path.expanduser(mra)
sys.path.append(os.path.join(MRA_LOC, '..'))
sys.path.append(MRA_LOC)

from src.DispatchManager import DispatchRunner
from ravenframework import ROMExternal
from ravenframework.utils import xmlUtils

/home/sotogj/.conda/envs/raven_libraries/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sotogj/.conda/envs/raven_libraries/lib/python3.10/site-packages/ray/_private/parameter.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/home/sotogj/.conda/envs/raven_libraries/lib/python3.10/site-packages/pkg_resources/__init__.py:2832: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
2024-10-02 06:52:41,226	INFO util.py:159 -- Missing pack

In [2]:
from HERON.src.dispatch.MRA_testing.utils.process_files import (get_heron_lib,
                                                                grab_sources,
                                                                populate_raven_dict)

from multiresolutionanalysis.notebooks.dispatch.utils.parsing_results import (fix_path,
                                                                              check_dir,
                                                                              check_and_make_dir)

from HERON.src.dispatch.MRA_testing.utils.modify_XMLs import modify_node_text

# Loop through Cases

In [3]:
scratch_dir = check_dir( '/scratch/sotogj/MRADispatch/NYISO/dispatch' )
CASES = [
    'Case1',
    'Case2a','Case2b',
    'Case3a','Case3b','Case3c',
    'Case4a','Case4b','Case4c',
    'Case5a','Case5d',
]

SOLVERS = ['ipopt', 'cbc', 'glpk', 'cplex_direct']

In [4]:
CASE_INFO = {}
CASE_INFO['ROM']     = 'fGOLD'
CASE_INFO['METRIC']  = 'NPV_C'
CASE_INFO['SAMPLES'] = 1

for SOLVER in SOLVERS:
    for CASE in CASES:

        # define case
        CASE_INFO['CASE']  = CASE

        # define input and output locations for file transfer
        dirname = f'dispatchOpt_{SOLVER}_W8760'
        MRA_scripts_dir = check_dir( os.path.join(mra, 'output/DISPATCH_COLLECT') )
        MRA_case_dir    = check_dir( os.path.join(scratch_dir, CASE_INFO['ROM'], CASE) )
        MRA_test_dir    = check_dir( os.path.join(MRA_case_dir, dirname) )
        if not MRA_test_dir:
            MRA_test_dir = os.path.join(MRA_case_dir, dirname)
            check_and_make_dir(MRA_test_dir)

        # define any modification methods for XML prior to running
        MODS = []
        MODS.append( partial(modify_node_text, address='./Case/dispatcher/pyomo/solver', val=SOLVER) )

        # run HERON and get HERON lib file location
        removeCapex = True
        HERON_lib = get_heron_lib(HERON_LOC,
                                MRA_scripts_dir,
                                MRA_test_dir,
                                CASE_INFO,
                                rerun=True,
                                removeCapex=removeCapex,
                                modifications=MODS)

        # create an external dispatch runner object, load HERON lib to it
        dispatch_runner = ExternalDispatchRunner()
        dispatch_runner.load_heron_lib(HERON_lib)

        # define macro and pivot parameter names
        macro_name = dispatch_runner._case.get_year_name()
        time_name  = dispatch_runner._case.get_time_name()

        # get all info from SOURCES (CSV, ARMA, etc.)
        SOURCES = grab_sources(dispatch_runner, macro_name, time_name)

        # create a raven_dict that acts as if it took all necessary info from the outer and the RAVEN part of the inner run
        raven_dict = populate_raven_dict(dispatch_runner, macro_name, time_name, SOURCES)

        # back to the Dispatch runner... make sure all RAVEN vars are saved and then run dispatch
        raven_vars = dispatch_runner.extract_variables_no_raven(raven_dict)

        start_time = time_mod.time()
        dispatch, metrics, tot_activity = dispatch_runner.run(raven_vars)
        end_time = time_mod.time()

        # collect all info from the run and dump it to a pickle file
        filename = f"{CASE}_{CASE_INFO['ROM']}_"
        filename += 'dNC_' if removeCapex else 'd_'
        filename += f"{CASE_INFO['METRIC']}"
        current_time = '2024_09_30' #time_mod.strftime("%Y_%m_%d", time_mod.localtime())
        collection_file = fix_path( os.path.join(MRA_test_dir,
                                                f'collect_{filename}__{current_time}.pk') )

        collection = {}
        collection['raven_dict'] = raven_dict
        collection['case_info']  = CASE_INFO
        collection['dispatch_runner'] = dispatch_runner
        collection['dispatch'] = dispatch
        collection['metrics'] = metrics
        collection['tot_activity'] = tot_activity
        collection['time_elapsed'] = end_time-start_time

        with open(collection_file, 'wb') as fp:
            pickle.dump(collection, fp)

NOT FOUND: /scratch/sotogj/MRADispatch/NYISO/dispatch/fGOLD/Case1/dispatchOpt_cplex_W8760
/scratch/sotogj/MRADispatch/NYISO/dispatch/fGOLD/Case1/dispatchOpt_cplex_W8760
NOT FOUND: /scratch/sotogj/MRADispatch/NYISO/dispatch/fGOLD/Case1/dispatchOpt_cplex_W8760/HERON_Case1__debugNoCap_from_NPV_C.xml
XML does not exist, copying over from /scratch/sotogj/MRADispatch/NYISO/dispatch/fGOLD/Case1/dispatchOpt_cplex_W8760
NOT FOUND: /scratch/sotogj/MRADispatch/NYISO/dispatch/fGOLD/Case1/dispatchOpt_cplex_W8760/heron.lib
Running HERON on /scratch/sotogj/MRADispatch/NYISO/dispatch/fGOLD/Case1/dispatchOpt_cplex_W8760/HERON_Case1__debugNoCap_from_NPV_C.xml


/home/sotogj/.conda/envs/raven_libraries/lib/python3.10/site-packages/pyutilib/misc/import_file.py:11: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


npp
market_electricity
source_electricity
sink_electricity
RUNNING HERON DISPATCH MANAGER
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 1.9611265659332275 s
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 1.893660306930542 s
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 1.9263575077056885 s
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 1.92

/home/sotogj/projects/TEAL/src/main.py:450: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  projCf[decomissionMask] += lifeCf[-1] * taxMult * np.power(inflRate, -1*years[decomissionMask])


npp
wind
solar
market_electricity
source_electricity
sink_electricity
RUNNING HERON DISPATCH MANAGER
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 2.684088945388794 s
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 2.8210244178771973 s
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 2.9241220951080322 s
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve

...

/home/sotogj/projects/TEAL/src/main.py:450: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  projCf[decomissionMask] += lifeCf[-1] * taxMult * np.power(inflRate, -1*years[decomissionMask])


reactor
turbine
tes
wind
solar
battery
market_electricity
htse
H2Tank
market_H2
source_H2
sink_H2
source_electricity
sink_electricity
source_thermal
sink_thermal
RUNNING HERON DISPATCH MANAGER
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 22.04812002182007 s
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 20.02135992050171 s
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 16.123356819152832 s
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG

/home/sotogj/projects/TEAL/src/main.py:450: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  projCf[decomissionMask] += lifeCf[-1] * taxMult * np.power(inflRate, -1*years[decomissionMask])


reactor
turbine
tes
wind
solar
battery
market_electricity
htse
H2Tank
market_H2
source_H2
sink_H2
source_electricity
sink_electricity
source_thermal
sink_thermal
RUNNING HERON DISPATCH MANAGER
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 14.802907466888428 s
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 15.05112886428833 s
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 14.325659990310669 s
Start: 0 End: 8760
DEBUGG using solver: cplex
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUG

/home/sotogj/projects/TEAL/src/main.py:450: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  projCf[decomissionMask] += lifeCf[-1] * taxMult * np.power(inflRate, -1*years[decomissionMask])
